In [1]:
import git
import pandas as pd
import datetime

pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv").to_csv("time_series_covid19_confirmed_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv").to_csv("time_series_covid19_confirmed_global.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv").to_csv("time_series_covid19_deaths_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv").to_csv("time_series_covid19_deaths_global.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv").to_csv("time_series_covid19_recovered_global.csv", index=False)

data_confirmed = pd.read_csv("time_series_covid19_confirmed_global.csv")
data_recovered = pd.read_csv("time_series_covid19_recovered_global.csv")
data_death     = pd.read_csv("time_series_covid19_deaths_global.csv")


data_salida_confirmed = pd.DataFrame() 
data_salida_recovered = pd.DataFrame()
data_salida_death = pd.DataFrame()
data_salida_confirmed["Province/State"] = data_confirmed["Province/State"] 
data_salida_confirmed["Country/Region"] = data_confirmed["Country/Region"] 
data_salida_confirmed["Lat"] = data_confirmed["Lat"]
data_salida_confirmed["Long"] = data_confirmed["Long"]

data_salida_recovered["Country/Region"] = data_recovered["Country/Region"] 
data_salida_recovered["Province/State"] = data_recovered["Province/State"] 
data_salida_death["Country/Region"] = data_death["Country/Region"]

columna_anterior = ""
for columna in data_confirmed.columns[4:len(data_confirmed)]:
    if(columna_anterior == ""):
        #print("no se hace nada")
        pass
    else:
        #print(columna + " y " + columna_anterior)
        
        data_salida_confirmed[columna] = data_confirmed[columna] - data_confirmed[columna_anterior]
        data_salida_recovered[columna] = data_recovered[columna] - data_recovered[columna_anterior]
        data_salida_death[columna]     = data_death[columna]     - data_death[columna_anterior]
        
    columna_anterior = columna
    
entrada = {"Province/State":""}   #,"Country/Region":"","Lat":"","Long":""}
salida = []
#index = 0
#data_salida = pd.DataFrame()

data_salida = pd.DataFrame(columns=data_salida_confirmed.columns[:4])

lista_confirmed = list(data_salida_confirmed.iterrows())
lista_recovered = list(data_salida_recovered.iterrows())
lista_death = list(data_salida_death.iterrows())

n = 0

for i in range(len(lista_confirmed)):
    dias_correlativo = 0
    flag = True
    for columna in data_salida_confirmed.columns[4:]:         
        #print(columna)
        #print(i[1][columna])
        entrada["Province/State"] = lista_confirmed[i][1]["Province/State"]
        entrada["Country/Region"] = lista_confirmed[i][1]["Country/Region"]
       
        entrada["Lat"] = lista_confirmed[i][1]["Lat"]
        entrada["Long"] = lista_confirmed[i][1]["Long"]
        formato = columna.split("/")
        entrada["fecha"] = datetime.date(int("20" + formato[2]),int(formato[0]),int(formato[1])).strftime("%d-%m-%Y")
        #print(lista_confirmed[i][1][columna]) 
        #print(entrada["Country/Region"])
        entrada["confirmados"] = lista_confirmed[i][1][columna]
        entrada["fallecidos"] = lista_death[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        
        if(flag):
            if(entrada["confirmados"] != 0):
                flag = False;
                dias_correlativo += 1
        else:
            dias_correlativo += 1
        entrada["dias correlativo"] = dias_correlativo
        #"""    
        #entrada["recuperados"] = lista_recovered[i][1][columna]
        salida.append(data_salida.append(entrada,ignore_index=True))
data_salida = pd.concat(salida)

entrada = {"Province/State":""}
salida = []
data_salida_aux = pd.DataFrame(columns=data_salida_confirmed.columns[:4])
for i in range(len(lista_recovered)):
    for columna in data_salida_recovered.columns[4:]:
        entrada["Province/State"] = lista_recovered[i][1]["Province/State"]
        entrada["Country/Region"] = lista_recovered[i][1]["Country/Region"]
        formato = columna.split("/")
        entrada["fecha"] = datetime.date(int(formato[2]+"20"),int(formato[0]),int(formato[1])).strftime("%d-%m-%Y")
        entrada["recuperado"] = lista_recovered[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        #print(entrada)
        #print("*****************************************************************")
        salida.append(data_salida_aux.append(entrada, ignore_index=True))
data_salida_aux = pd.concat(salida)
del data_salida_aux["Province/State"]
del data_salida_aux["Country/Region"]
del data_salida_aux["fecha"]
del data_salida_aux["Lat"]
del data_salida_aux["Long"]


merged_left = pd.merge(left=data_salida, right=data_salida_aux, how='left', left_on='codigo', right_on='codigo')
#merged_left


#merged_left.to_csv("acumuladoLuis.csv", index=False)
merged_left.to_csv("acumulado.csv", index=False)
#merged_left.to_csv("acumulado.csv", index=False)

repoLocal = git.Repo( 'C:/Users/limc_/Documents/GitHub/Datos' )
print(repoLocal.git.status())
repoLocal.git.add(".")
try:
    repoLocal.git.commit(m='Update automatico via python')
except:
    pass

origin = repoLocal.remote(name='origin')
origin.push()

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Actualizar.py
	modified:   Johns_Hopkins-covid19/series/SacarDatos.ipynb
	modified:   Johns_Hopkins-covid19/series/acumulado.csv
	modified:   Johns_Hopkins-covid19/series/time_series_covid19_confirmed_US.csv
	modified:   Johns_Hopkins-covid19/series/time_series_covid19_confirmed_global.csv
	modified:   Johns_Hopkins-covid19/series/time_series_covid19_deaths_global.csv
	modified:   Johns_Hopkins-covid19/series/time_series_covid19_recovered_global.csv
	modified:   testActualizar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
1
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
0
Afghanistan
3
Afghanistan
0
Afghanistan
1
Afghanistan
2
Afghanistan
0
Afghanistan
0
Afghanistan
4
Afghanistan
5
Afghanistan
5
Afghanistan
1
Afghanistan
0
Afghanistan
0
Afghanistan
2
Afghanistan
0
Afghanistan
16
Afghanistan
0
Afghanistan
34
Afghanistan
10
Afghanistan
10
Afghanistan
16
Afghanistan
0
Afghanistan
10
Afghanistan
50
Afghanistan
4
Afghanistan
63
Afghanistan
0
Albania
0


186
Australia
212
Australia
174
Australia
241
Australia
0
Australia
150
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
1
Australia
0
Australia
-1
Australia
0
Australia
0
Australia
0
Australia
1
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
2
Australia
0
Australia
2
Australia
0
Australia
1
Australia
0
Australia
6
Australia
0
Australia
3
Australia
0
Australia
0
Australia
2
Australia
2
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
0
Australia
1
Aus

0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
3
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
0
Bangladesh
2
Bangladesh
3
Bangladesh
2
Bangladesh
4
Bangladesh
3
Bangladesh
3
Bangladesh
5
Bangladesh
2
Bangladesh
6
Bangladesh
6
Bangladesh
0
Bangladesh
5
Bangladesh
4
Bangladesh
0
Bangladesh
0
Bangladesh
1
Bangladesh
2
Bangladesh
3
Bangladesh
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0
Barbados
0


18
Bulgaria
10
Bulgaria
1
Bulgaria
15
Bulgaria
25
Bulgaria
2
Bulgaria
33
Bulgaria
36
Bulgaria
24
Bulgaria
14
Bulgaria
17
Bulgaria
24
Bulgaria
22
Bulgaria
29
Bulgaria
38
Bulgaria
15
Bulgaria
13
Bulgaria
40
Bulgaria
23
Bulgaria
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina Faso
1
Burkina Faso
1
Burkina Faso
0
Burkina Faso
0
Burkina Faso
0
Burkina

0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
1
Canada
0
Canada
0
Canada
2
Canada
0
Canada
5
Canada
4
Canada
3
Canada
2
Canada
0
Canada
2
Canada
3
Canada
3
Canada
1
Canada
5
Canada
2
Canada
5
Canada
1
Canada
32
Canada
5
Canada
25
Canada
73
Canada
8
Canada
36
Canada
36
Canada
51
Canada
69
Canada
48
Canada
78
Canada
85
Canada
100
Canada
170
Canada
136
Canada
150
Canada
211
Canada
351
Canada
260
Canada
426
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
1
Canada
0
Canada
0


...

3
Thailand
6
Thailand
11
Thailand
5
Thailand
7
Thailand
32
Thailand
33
Thailand
30
Thailand
35
Thailand
60
Thailand
50
Thailand
89
Thailand
188
Thailand
122
Thailand
106
Thailand
107
Thailand
111
Thailand
91
Thailand
109
Thailand
143
Thailand
136
Thailand
127
Thailand
120
Thailand
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
1
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
0
Togo
8
Togo
7
Togo
0
Togo
2
Togo
2
Togo
3
Togo
0
Togo
2
Togo
0
Togo
0
Togo
5
Togo
4
Togo
2
Togo
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinidad and Tobago
0
Trinid

0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
1
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
2
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
2
United Kingdom
1
United Kingdom
2
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
4
United Kingdom
2
United Kingdom
8
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingd

3
Uzbekistan
4
Uzbekistan
10
Uzbekistan
15
Uzbekistan
13
Uzbekistan
16
Uzbekistan
40
Uzbekistan
5
Uzbekistan
23
Uzbekistan
9
Uzbekistan
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
0
Venezuela
2
Venezuela
8
Venezuela
7
Venezuela
16
Venezuela
3
Venezuela
6
Venezuela
0
Venezuela
28
Venezuela
0
Venezuela
7
Venezuela
7
Venezuela
7
Venezuela
16
Venezuela
0
Venezuela
12
Venezuela
0
Venezuela
16
Venezuela
0
Venezuela
8
Venezuela
2
Vietnam
0
Vietnam

0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Canada
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
0
Laos
2
Laos
1
Laos
3

0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
0
Burma
8
Burma
0
Burma
2
Burma
4
Burma
1
Burma
0
Burma
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdom
0
United Kingdo

In [5]:
#print(data_salida_recovered[data_salida_recovered["Country/Region"] == "Chile"])
#data_salida_recovered[data_salida_recovered["Country/Region"] == "Chile"] 
#print(data_recovered[data_recovered["Country/Region"] == "Chile"])
#data_recovered
#merged_left[merged_left["Country/Region"] == "Chile"].head()

merged_left.to_excel("acumuladoTest.xlsx", index=False)

In [33]:
data_salida.shape, data_salida_aux.shape, merged_left.shape

((17408, 8), (15730, 2), (17408, 9))

In [77]:
data_salida_aux[data_salida_aux["codigo"] == "Chilenan30-03-2020"]

,codigo,recuperado
0,Chilenan30-03-2020,81.0


In [51]:
lista_recovered[0][1]

Country/Region    Afghanistan
1/24/20                     0
1/25/20                     0
1/26/20                     0
1/27/20                     0
                     ...     
3/27/20                     0
3/28/20                     0
3/29/20                     0
3/30/20                     0
3/31/20                     3
Name: 0, Length: 69, dtype: object

In [84]:
data_salida_confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/23/20', '1/24/20',
       '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20',
       '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20',
       '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20',
       '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20',
       '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20',
       '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
       '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20',
       '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20',
       '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20',
       '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20'],
      dtype='object')

In [60]:
type(lista_confirmed[1][1]["3/13/20"])

int

In [85]:
data_test = pd.read_csv("acumulado.csv")
data_test[data_test["Country/Region"] == "Chile"].tail()

,Province/State,Country/Region,Lat,Long,codigo,confirmados,fallecidos,fecha,recuperado
3425,NaN,Chile,-35.6751,-71.543,Chilenan28-03-2020,299.0,1.0,28-03-2020,18.0
3426,NaN,Chile,-35.6751,-71.543,Chilenan29-03-2020,230.0,1.0,29-03-2020,14.0
3427,NaN,Chile,-35.6751,-71.543,Chilenan30-03-2020,310.0,1.0,30-03-2020,81.0
3428,NaN,Chile,-35.6751,-71.543,Chilenan31-03-2020,289.0,4.0,31-03-2020,0.0
3429,NaN,Chile,-35.6751,-71.543,Chilenan01-04-2020,293.0,4.0,01-04-2020,78.0


In [2]:
import pandas as pd
import datetime
data_old =pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
data_old2=  pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,29,32,37,45,51,55,56,58,69,77
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,0,0,0,0,0,0,0,0,2,6
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,31,39,51,64,79,100,127,174,239,286
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,7,17,17,17,19,22,0,0,30,30


In [7]:
for i in lista_confirmed:
    print(i)

(0, Province/State            NaN
Country/Region    Afghanistan
Lat                        33
Long                       65
1/23/20                     0
                     ...     
3/28/20                     0
3/29/20                    10
3/30/20                    50
3/31/20                     4
4/1/20                     63
Name: 0, Length: 74, dtype: object)
(1, Province/State        NaN
Country/Region    Albania
Lat               41.1533
Long              20.1683
1/23/20                 0
                   ...   
3/28/20                11
3/29/20                15
3/30/20                11
3/31/20                20
4/1/20                 16
Name: 1, Length: 74, dtype: object)
(2, Province/State        NaN
Country/Region    Algeria
Lat               28.0339
Long               1.6596
1/23/20                 0
                   ...   
3/28/20                45
3/29/20                57
3/30/20                73
3/31/20               132
4/1/20                131
Name: 2, Lengt

Name: 76, Length: 74, dtype: object)
(77, Province/State    Tianjin
Country/Region      China
Lat               39.3054
Long              117.323
1/23/20                 0
                   ...   
3/28/20                 6
3/29/20                 5
3/30/20                 8
3/31/20                 0
4/1/20                  2
Name: 77, Length: 74, dtype: object)
(78, Province/State      Tibet
Country/Region      China
Lat               31.6927
Long              88.0924
1/23/20                 0
                   ...   
3/28/20                 0
3/29/20                 0
3/30/20                 0
3/31/20                 0
4/1/20                  0
Name: 78, Length: 74, dtype: object)
(79, Province/State    Xinjiang
Country/Region       China
Lat                41.1129
Long               85.2401
1/23/20                  2
                    ...   
3/28/20                  0
3/29/20                  0
3/30/20                  0
3/31/20                  0
4/1/20                   0
Name:

Name: 184, Length: 74, dtype: object)
(185, Province/State        NaN
Country/Region      Qatar
Lat               25.3548
Long              51.1839
1/23/20                 0
                   ...   
3/28/20                28
3/29/20                44
3/30/20                59
3/31/20                88
4/1/20                 54
Name: 185, Length: 74, dtype: object)
(186, Province/State        NaN
Country/Region    Romania
Lat               45.9432
Long              24.9668
1/23/20                 0
                   ...   
3/28/20               160
3/29/20               363
3/30/20               294
3/31/20               136
4/1/20                215
Name: 186, Length: 74, dtype: object)
(187, Province/State       NaN
Country/Region    Russia
Lat                   60
Long                  90
1/23/20                0
                   ...  
3/28/20              228
3/29/20              270
3/30/20              302
3/31/20              501
4/1/20               440
Name: 187, Length: 74